# 📊 EXPORTAÇÃO DE DADOS PARA POWER BI

**Objetivo:** Preparar e gerar todos os datasets necessários para criação de dashboards de análise de crédito no Power BI.

### ⏳ Arquivos gerados
- dataset_powerbi.csv – dataset principal com probabilidades e previsões
- threshold_analysis.csv – métricas para diferentes thresholds
- roc_curve.csv – dados para curva ROC de cada modelo
- precision_recall_curve.csv – dados para curva Precision-Recall
- feature_importance_comparison.csv – importância das features comparando LR e RF
- confusion_matrix.csv – matrizes de confusão por threshold
- model_comparison.csv – comparação de métricas entre modelos
- inadimplencia_ocupacao.csv – taxa de inadimplência por ocupação
- inadimplencia_score.csv – taxa de inadimplência por faixa de score
- inadimplencia_renda.csv – taxa de inadimplência por faixa de renda

---
# ⭐ MODELAGEM e AVALIAÇÃO DE MODELOS
---

In [ ]:
# CÉLULA 1. IMPORTAÇÃO DE BIBLIOTECAS
import pandas as pd
import numpy as np
import sqlite3
import os

# ML
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import *
from sklearn.model_selection import train_test_split

# Criar pasta de destino
os.makedirs('../powerbi', exist_ok=True)

print("📊 INICIANDO EXPORTAÇÃO...")

In [ ]:
# CÉLULA 2. CARREGANDO DADOS DO SQLITE
conn = sqlite3.connect('../data/database.db')
df = pd.read_sql("SELECT * FROM clientes", conn)
conn.close()

print(f"✓ Dados carregados: {len(df):,} registros")

# Criar feature renda_per_capita
df['renda_per_capita'] = df['renda_anual'] / (df['numero_dependentes'] + 1)
print(f"✓ Feature 'renda_per_capita' criada")

In [ ]:
# CÉLULA 3. PREPARANDO DADOS PARA MODELAGEM
target = 'inadimplente'
features = [col for col in df.columns if col not in ['inadimplente', 'id_cliente']]

X = df[features].copy()
y = df[target]

# Split treino/teste
X_train, X_test, y_train, y_test = train_test_split(
  X, y, test_size=0.3, random_state=42, stratify=y
)

print(f"✓ Split: {len(X_train):,} treino / {len(X_test):,} teste")

In [ ]:
# CÉLULA 4. ENCODING DE VARIÁVEIS CATEGÓRICAS
cat_cols = X.select_dtypes(include='object').columns
encoders = {}

for col in cat_cols:
  le = LabelEncoder()
  X_train[col] = le.fit_transform(X_train[col])
  X_test[col] = le.transform(X_test[col])
  encoders[col] = le

print(f"✓ Encoding aplicado em {len(cat_cols)} colunas")

# Normalização
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print(f"✓ Normalização aplicada")

In [ ]:
# CÉLULA 5. TREINAR LR (LOGISTIC REGRESSION)
print("\n🎯 Treinando Logistic Regression...")
lr = LogisticRegression(random_state=42, max_iter=1000)
lr.fit(X_train_scaled, y_train)
y_proba_lr = lr.predict_proba(X_test_scaled)[:, 1]
y_pred_lr = lr.predict(X_test_scaled)

print(f"  AUC: {roc_auc_score(y_test, y_proba_lr):.4f}")
print(f"  Recall: {recall_score(y_test, y_pred_lr):.4f}")

In [ ]:
# CÉLULA 6. TREINANDO RF (RANDOM FOREST)
print("\n🌲 Treinando Random Forest...")
rf = RandomForestClassifier(
    n_estimators=100,
    max_depth=10,
    class_weight='balanced',
    random_state=42,
    n_jobs=-1
)
rf.fit(X_train_scaled, y_train)
y_proba_rf = rf.predict_proba(X_test_scaled)[:, 1]
y_pred_rf = rf.predict(X_test_scaled)

print(f"AUC: {roc_auc_score(y_test, y_proba_rf):.4f}")
print(f"Recall: {recall_score(y_test, y_pred_rf):.4f}") 

---
# 📦 EXPORTAR DATASETS PARA POWER BI
---

In [ ]:
# CÉLULA 7. DATASET PRINCIPAL
print("📦 EXPORTANDO DATASETS PARA ../powerbi/")

print("\n1️⃣ Dataset principal")
df_export = df.copy()
df_export['probabilidade_lr'] = np.nan
df_export['probabilidade_rf'] = np.nan
df_export['predicao_lr_50'] = np.nan
df_export['predicao_rf_50'] = np.nan
df_export['conjunto'] = 'treino'

test_indices = X_test.index
df_export.loc[test_indices, 'probabilidade_lr'] = y_proba_lr
df_export.loc[test_indices, 'probabilidade_rf'] = y_proba_rf
df_export.loc[test_indices, 'predicao_lr_50'] = (y_proba_lr >= 0.5).astype(int)
df_export.loc[test_indices, 'predicao_rf_50'] = (y_proba_rf >= 0.5).astype(int)
df_export.loc[test_indices, 'conjunto'] = 'teste'

df_export.to_csv('../powerbi/dataset_powerbi.csv', index=False)
print(f"✓ dataset_powerbi.csv ({len(df_export):,} registros)")

In [ ]:
# CÉLULA 8. ANÁLISE DE THRESHOLD
print("2️⃣ Análise de threshold")
threshold_data = []

for t in np.arange(0.1, 0.95, 0.05):
  y_pred = (y_proba_lr >= t).astype(int)
  threshold_data.append({
    'threshold': t,
    'precision': precision_score(y_test, y_pred),
    'recall': recall_score(y_test, y_pred),
    'f1_score': f1_score(y_test, y_pred),
    'accuracy': accuracy_score(y_test, y_pred)
  })

df_threshold = pd.DataFrame(threshold_data)
df_threshold.to_csv('../powerbi/threshold_analysis.csv', index=False)
print(f"✓ threshold_analysis.csv ({len(df_threshold)} thresholds)")

In [ ]:
# CÉLULA 9. CURVA ROC
print("3️⃣ Curva ROC")
fpr_lr, tpr_lr, _ = roc_curve(y_test, y_proba_lr)
fpr_rf, tpr_rf, _ = roc_curve(y_test, y_proba_rf)
roc_data = []

for i in range(len(fpr_lr)):
  roc_data.append({'modelo': 'Logistic Regression', 'fpr': fpr_lr[i], 'tpr': tpr_lr[i]})
for i in range(len(fpr_rf)):
  roc_data.append({'modelo': 'Random Forest', 'fpr': fpr_rf[i], 'tpr': tpr_rf[i]})

df_roc = pd.DataFrame(roc_data)
df_roc.to_csv('../powerbi/roc_curve.csv', index=False)
print(f"✓ roc_curve.csv ({len(df_roc)} pontos)")

In [ ]:
# CÉLULA 10. PRECISION-RECALL CURVE
print("4️⃣ PRC")
precision_lr, recall_lr, _ = precision_recall_curve(y_test, y_proba_lr)
precision_rf, recall_rf, _ = precision_recall_curve(y_test, y_proba_rf)
pr_data = []

for i in range(len(precision_lr)):
  pr_data.append({'modelo': 'Logistic Regression', 'precision': precision_lr[i], 'recall': recall_lr[i]})
for i in range(len(precision_rf)):
  pr_data.append({'modelo': 'Random Forest', 'precision': precision_rf[i], 'recall': recall_rf[i]})

df_pr = pd.DataFrame(pr_data)
df_pr.to_csv('../powerbi/precision_recall_curve.csv', index=False)
print(f"✓ precision_recall_curve.csv ({len(df_pr)} pontos)")

In [ ]:
# CÉLULA 11. FEATURE IMPORTANCE
print("\n5️⃣ Feature Importance")
feature_names = X_train.columns

df_importance = pd.DataFrame({
  'feature': feature_names,
  'importance_lr': np.abs(lr.coef_[0]),
  'importance_rf': rf.feature_importances_,
  'coefficient_lr': lr.coef_[0]
}).sort_values('importance_lr', ascending=False)

df_importance.to_csv('../powerbi/feature_importance_comparison.csv', index=False)
print(f"✓ feature_importance_comparison.csv ({len(df_importance)} features)")

In [ ]:
# CÉLULA 12. MATRIZ DE CONFUSÃO
print("6️⃣ Matrizes de confusão")
conf_data = []
thresholds_conf = [0.30, 0.40, 0.50, 0.60, 0.70]

for t in thresholds_conf:
  y_pred = (y_proba_lr >= t).astype(int)
  cm = confusion_matrix(y_test, y_pred)

  conf_data.append({'threshold': t, 'categoria': 'TN (Verdadeiro Negativo)', 'valor': cm[0,0], 'percentual': cm[0,0] / cm.sum()})
  conf_data.append({'threshold': t, 'categoria': 'FP (Falso Positivo)', 'valor': cm[0,1], 'percentual': cm[0,1] / cm.sum()})
  conf_data.append({'threshold': t, 'categoria': 'FN (Falso Negativo)', 'valor': cm[1,0], 'percentual': cm[1,0] / cm.sum()})
  conf_data.append({'threshold': t, 'categoria': 'TP (Verdadeiro Positivo)', 'valor': cm[1,1], 'percentual': cm[1,1] / cm.sum()})

df_confusion = pd.DataFrame(conf_data)
df_confusion.to_csv('../powerbi/confusion_matrix.csv', index=False)
print(f"✓ confusion_matrix.csv ({len(df_confusion)} registros)")

In [ ]:
# CÉLULA 13. COMPARAÇÃO DE MODELOS
print("7️⃣ Comparação LR vs RF")
modelo_comp = []

for threshold in [0.30, 0.40, 0.50, 0.60]:
  y_pred_lr = (y_proba_lr >= threshold).astype(int)
  y_pred_rf = (y_proba_rf >= threshold).astype(int)
    
  modelo_comp.append({
      'modelo': 'Logistic Regression',
      'threshold': threshold,
      'accuracy': accuracy_score(y_test, y_pred_lr),
      'precision': precision_score(y_test, y_pred_lr),
      'recall': recall_score(y_test, y_pred_lr),
      'f1_score': f1_score(y_test, y_pred_lr),
      'auc_roc': roc_auc_score(y_test, y_proba_lr)
  })
    
  modelo_comp.append({
      'modelo': 'Random Forest',
      'threshold': threshold,
      'accuracy': accuracy_score(y_test, y_pred_rf),
      'precision': precision_score(y_test, y_pred_rf),
      'recall': recall_score(y_test, y_pred_rf),
      'f1_score': f1_score(y_test, y_pred_rf),
      'auc_roc': roc_auc_score(y_test, y_proba_rf)
  })

df_modelo_comp = pd.DataFrame(modelo_comp)
df_modelo_comp.to_csv('../powerbi/model_comparison.csv', index=False)
print(f"✓ model_comparison.csv ({len(df_modelo_comp)} registros)")

In [ ]:
# CÉLULA 14. ESTATÍSTICAS POR GRUPO
print("8️⃣ Estatísticas por grupo...")

# Inadimplência por ocupação
inad_ocupacao = df.groupby('ocupacao')['inadimplente'].agg(['mean', 'count']).reset_index()
inad_ocupacao.columns = ['ocupacao', 'taxa_inadimplencia', 'total']
inad_ocupacao.to_csv('../powerbi/inadimplencia_ocupacao.csv', index=False)

# Inadimplência por score serasa
df['faixa_score'] = pd.cut(df['score_serasa_externo'], 
                            bins=[0, 400, 500, 600, 700, 1000],
                            labels=['0-400', '400-500', '500-600', '600-700', '700+'])
inad_score = df.groupby('faixa_score', observed=True)['inadimplente'].agg(['mean', 'count']).reset_index()
inad_score.columns = ['faixa_score', 'taxa_inadimplencia', 'total']
inad_score.to_csv('../powerbi/inadimplencia_score.csv', index=False)

# Inadimplência por faixa de renda
df['faixa_renda'] = pd.cut(df['renda_anual'], 
                            bins=[0, 30000, 60000, 100000, 1000000],
                            labels=['<30k', '30-60k', '60-100k', '>100k'])
inad_renda = df.groupby('faixa_renda', observed=True)['inadimplente'].agg(['mean', 'count']).reset_index()
inad_renda.columns = ['faixa_renda', 'taxa_inadimplencia', 'total']
inad_renda.to_csv('../powerbi/inadimplencia_renda.csv', index=False)

print(f"✓ inadimplencia_ocupacao.csv")
print(f"✓ inadimplencia_score.csv")
print(f"✓ inadimplencia_renda.csv")

---
# ✅ EXPORTAÇÃO CONCLUÍDA COM SUCESSO!
--- 

In [1]:
print("Arquivos criados em ../powerbi/:")
print("1. dataset_powerbi.csv")
print("2. threshold_analysis.csv")
print("3. roc_curve.csv")
print("4. precision_recall_curve.csv")
print("5. feature_importance_comparison.csv")
print("6. confusion_matrix.csv")
print("7. model_comparison.csv")
print("8. inadimplencia_ocupacao.csv")
print("9. inadimplencia_score.csv")
print("10. inadimplencia_renda.csv")

Arquivos criados em ../powerbi/:
1. dataset_powerbi.csv
2. threshold_analysis.csv
3. roc_curve.csv
4. precision_recall_curve.csv
5. feature_importance_comparison.csv
6. confusion_matrix.csv
7. model_comparison.csv
8. inadimplencia_ocupacao.csv
9. inadimplencia_score.csv
10. inadimplencia_renda.csv
